# Need Project: Feature Union and Model Generator using Heterogeneous Dataset

In [607]:
#python 3
import pandas as pd
# from sklearn.cross_validation import train_test_split

from sklearn.model_selection import train_test_split

#model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection._split import StratifiedShuffleSplit
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.svm.classes import SVC, LinearSVC


import nltk
import numpy as np


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score

# Constants and read csv file

In [608]:

file_name = 'needs_tweets_eng_processed_all_regions.csv'
test_size=0.33
#All needs
genres = ['water', 'volunteer', 'bottle', 'emergency', 'clothes', 'grocery', 'power', 'boat', 'gas', 'fund', 'store', 'food', 'money', 'contact', 'community', 'dog', 'rescue', 'school', 'nurse', 'assistance', 'car', 'advisory', 'home', 'donate', 'safe', 'relief', 'damage', 'oil', 'support', 'text', 'beer', 'family', 'trail', 'victim', 'tank', 'cable', 'shelter', 'truck', 'donation', 'service', 'flood', 'call', 'wine', 'supply', 'need', 'people', 'help', 'restaurant']
#min needs freq is 10
genres = ['water', 'volunteer', 'emergency', 'clothes', 'grocery', 'power', 'boat', 'gas', 'fund', 'store', 'food', 'money', 'contact', 'community', 'dog', 'rescue', 'school', 'nurse', 'assistance', 'car', 'advisory', 'home', 'donate', 'safe', 'relief', 'damage', 'oil', 'support', 'text', 'beer', 'family', 'trail', 'victim', 'tank', 'shelter', 'truck', 'donation', 'service', 'flood', 'call', 'wine', 'supply', 'need', 'people', 'help', 'restaurant']
#min needs freq is 50
genres = ['family', 'shelter', 'volunteer', 'donation', 'service', 'store', 'food', 'rescue', 'community', 'flood', 'call', 'school', 'nurse', 'home', 'donate', 'safe', 'relief', 'need', 'people', 'support', 'help', 'water']


random_state = 37

data_df = pd.read_csv(file_name, delimiter='\t')
data_df.head()
len(data_df)

2317

# Combine the features raw data : hour, region_name, tweet_text. This is used later for feature union

In [610]:
x_column_name = ['region_name', 'hour', 'tweet_text']
combine_seperator = '\n\n'
number_of_features = 3
data_df['combined_tweet_text']=data_df['hour'].astype(str) + combine_seperator +data_df['region_name'] + combine_seperator + data_df['tweet_text']
data_df.head()
# data_df['combined_tweet_text'][0].split(combine_seperator)

,tweet_text,region_name,multi_class_label,hour,multi_class_label_vec,combined_tweet_text
0,"Models need to eat too. ""Always Hungry"" thank...",Willowbrook,need,0,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","0\n\nWillowbrook\n\nModels need to eat too. ""..."
1,We're #hiring! Click to apply: Service Technic...,Fourth Ward,service,1,"0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1\n\nFourth Ward\n\nWe're #hiring! Click to ap...
2,Me and Bae paid our support to my cuz tonight ...,Independence Heights,support,1,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1\n\nIndependence Heights\n\nMe and Bae paid o...
3,Wings Over Water - Avenida Houston - Downtown ...,Mid West,water,4,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",4\n\nMid West\n\nWings Over Water - Avenida Ho...
4,Up to 54% Off Haircut Packages at Paul Mitchel...,Independence Heights,school,5,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0...",5\n\nIndependence Heights\n\nUp to 54% Off Hai...


# Split dataset

In [611]:


data_x = data_df['combined_tweet_text'].as_matrix()
data_y = data_df['multi_class_label_vec'].as_matrix()

stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=test_size, random_state=random_state)

x_train, x_test, y_train, y_test = train_test_split(data_x,data_y,test_size=test_size, random_state=random_state)



len(genres)
# data_x

# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x.strip() for x in x_train.tolist()]

test_x = [x.strip() for x in x_test.tolist()]
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
len(train_x)

/Users/habugell/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/habugell/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/habugell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1552

# Define Feature Union and classifier - classes and pipelines

In [670]:
# return one column of the x_train according to the key pass (hour, region_name, tweet_text)
class ItemSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

#class to split the combined x_train to 3 columns hour, region_name, tweet_text according to the key passed
class keyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        #hour, region_name, tweet_text
        features = np.recarray(shape=(len(posts),),
                               dtype=[('hour', object), ('region_name', object), ('tweet_text', object)])
        for i, text in enumerate(posts):
            combined_text = text.split(combine_seperator)
            assert len(combined_text) == number_of_features
             
            features['hour'][i] = combined_text[0]

            features['region_name'][i] = combined_text[1]

            features['tweet_text'][i] = combined_text[2]

        return features

#Feature calc method to get the length of the tweet and the number of dots.
class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]

#define a pipline: first keyExtractor (to split the combined columns).
#    Then perform Feature Union for hour, region_name, tweet_text columns.
#    Finally, run the classifier to select the model using OneVsRest Classifier method with wieghts  
#             for for hour, region_name, and tweet_text, tweet text's stats, repectively.
pipeline_feature_union = Pipeline([
    # Extract the hour, region_name, tweet_text 
    ('hour_region_name_tweet_text', keyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for TFIDF features from the tweets's texts
            ('tweet_text', Pipeline([
                ('selector', ItemSelector(key='tweet_text')),
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
#                 ('tfidf', CountVectorizer(stop_words=stop_words)),
#                 ('transf', TfidfTransformer()),

            ])),

            # Pipeline for text statistics features from the tweets's texts
            ('tweet_text_stats', Pipeline([
                ('selector', ItemSelector(key='tweet_text')),
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
                
            ])),


            # Pipeline for TFIDF features from the tweets's region names
            ('region_name', Pipeline([
                ('selector', ItemSelector(key='region_name')),
#                 ('tfidf', TfidfVectorizer(strip_accents='ascii', stop_words=stop_words)),
                ('tfidf', TfidfVectorizer(strip_accents='ascii', stop_words=stop_words)),

            ])),

            # Pipeline for TFIDF features from the tweets's hours (24)
            ('hour', Pipeline([
                ('selector', ItemSelector(key='hour')),
#                 ('tfidf', TfidfVectorizer(strip_accents='ascii', stop_words=stop_words)),
                ('tfidf', TfidfVectorizer(strip_accents='ascii', stop_words=stop_words)),

            ])),

        ],

        # weight components in FeatureUnion
#         transformer_weights={
#             'tweet_text': 1.0,
#             'region_name': 0.25,
#             'hour': 0.25,
#         },

        transformer_weights={
            'tweet_text': 0.90,#1.0
            'tweet_text_stats': 0.01, #0.01
            'region_name': 0.25, #0.5
            'hour': 0.25, #0.5
        },

    
    )),

    # Use a SVC classifier on the combined features
#     ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
    ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=10)),

    #SVC(kernel='linear')
#     ('clf', OneVsRestClassifier(SVC(kernel='linear'), n_jobs=1)),

#     ('clf', OneVsOneClassifier(LinearSVC(), n_jobs=1)),

])



In [671]:
# #Check the list of available parameters with `estimator.get_params().keys()
# grid.estimator.get_params().keys()


# Build the model using specific parameters.

In [672]:
print('OneVsAll')
#this takes a couple of minutes
def grid_search_feature_union(train_x, train_y, test_x, test_y, genres, parameters, pipeline):
    print('####start, grid_search_feature_union')
#     grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=3, verbose=10)
    #use k-folds for cross-validation
#     grid_search_tune = GridSearchCV(pipeline, parameters, cv=5, n_jobs=3, verbose=2)
#     grid_search_tune = GridSearchCV(pipeline, parameters, cv=5, n_jobs=10, verbose=10)
    grid_search_tune = GridSearchCV(pipeline, parameters, cv=5, n_jobs=1, verbose=10)



    print('####Step 1, grid_search_feature_union')

    grid_search_tune.fit(train_x, train_y)
    print('####Step 2, grid_search_feature_union')


    print()
    print("Best parameters set:")
    print(grid_search_tune.best_estimator_.steps)
    print()

    # measuring performance on test set
    print("Applying best classifier on test data:")
    best_clf = grid_search_tune.best_estimator_
    predictions = best_clf.predict(test_x)

    print('################ test_y[0]', test_y[0], 'predictions[0]', predictions[0])
    print(classification_report(test_y, predictions, target_names=genres))
    # Get the model accuracy
    print('The model accuracy: ', accuracy_score(test_y, predictions))
    
    return grid_search_tune, best_clf, predictions


parameters = {
    #hour, region_name, tweet_text
#     'union__tweet_text__tfidf__max_df': (0.25, 0.5, 0.75),
#     'union__tweet_text__tfidf__min_df': (2, 3),
    'union__tweet_text__tfidf__max_df': (0.25, 0.5),
#     'union__tweet_text__tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'union__tweet_text__tfidf__ngram_range': [(1, 1)],


#     'union__region_name__tfidf__min_df': (2, 3),
    'union__region_name__tfidf__max_df': (0.25, 0.5),
    'union__region_name__tfidf__ngram_range': [(1, 1)],

#     'union__hour__tfidf__min_df': (2, 3),
    'union__hour__tfidf__max_df': (0.25, 0.5),
    'union__hour__tfidf__ngram_range': [(1, 1)],



#     'clf__C': [0.1, 1, 10, 100],
    "clf__estimator__C": [0.01, 0.1],
    "clf__estimator__class_weight": ['balanced', None], 
    #None
#     "clf__estimator__class_weight": [None, None], 

    #clf__estimator__max_iter
    
     }

grid, best_clf, predictions = grid_search_feature_union(train_x, y_train, test_x, y_test, genres, parameters, pipeline_feature_union)

OneVsAll
####start, grid_search_feature_union
####Step 1, grid_search_feature_union
Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] clf__estimator__C=0.01, clf__estimator__class_weight=balanced, union__hour__tfidf__max_df=0.25, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.25, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1) 


/Users/habugell/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[CV]  clf__estimator__C=0.01, clf__estimator__class_weight=balanced, union__hour__tfidf__max_df=0.25, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.25, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1), score=0.6581632653061225, total=   0.4s
[CV] clf__estimator__C=0.01, clf__estimator__class_weight=balanced, union__hour__tfidf__max_df=0.25, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.25, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  clf__estimator__C=0.01, clf__estimator__class_weight=balanced, union__hour__tfidf__max_df=0.25, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.25, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1), score=0.7697160883280757, total=   0.5s
[CV] clf__estimator__C=0.01, clf__estimator__class_weight=balanced, union__hour__tfidf__max_df=0.25, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.25, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1) 


...

[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, union__hour__tfidf__max_df=0.25, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.5, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.5, union__tweet_text__tfidf__ngram_range=(1, 1), score=0.8676470588235294, total=   0.4s
[CV] clf__estimator__C=0.1, clf__estimator__class_weight=None, union__hour__tfidf__max_df=0.5, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.25, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1) 
[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, union__hour__tfidf__max_df=0.5, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.25, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1), score=0.6556122448979592, total=   0.4

[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, union__hour__tfidf__max_df=0.5, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.5, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1), score=0.8395904436860068, total=   0.4s
[CV] clf__estimator__C=0.1, clf__estimator__class_weight=None, union__hour__tfidf__max_df=0.5, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.5, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1) 
[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, union__hour__tfidf__max_df=0.5, union__hour__tfidf__ngram_range=(1, 1), union__region_name__tfidf__max_df=0.5, union__region_name__tfidf__ngram_range=(1, 1), union__tweet_text__tfidf__max_df=0.25, union__tweet_text__tfidf__ngram_range=(1, 1), score=0.8848920863309353, total=   0.4s


[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:  1.5min finished


####Step 2, grid_search_feature_union

Best parameters set:
[('hour_region_name_tweet_text', keyExtractor()), ('union', FeatureUnion(n_jobs=1,
       transformer_list=[('tweet_text', Pipeline(memory=None,
     steps=[('selector', ItemSelector(key='tweet_text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5,...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None))]))],
       transformer_weights={'tweet_text': 0.9, 'tweet_text_stats': 0.01, 'region_name': 0.25, 'hour': 0.25})), ('clf', OneVsRestClassifier(estimator=LinearSVC(C=0.1, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=10))]

Applying best classifier on test data:
###########

/Users/habugell/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 109, does not match size of target_names, 22
  .format(len(labels), len(target_names))
/Users/habugell/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/habugell/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [673]:
# pipeline_feature_union

In [674]:
print('y_train', len(y_train))

print('y_test', len(y_test))



y_train 1552
y_test 765


In [675]:
# from sklearn.metrics import multilabel_confusion_matrix
# multilabel_confusion_matrix(y_test, predictions, labels=genres)


In [676]:
print(classification_report(y_test, predictions, target_names=genres))

             precision    recall  f1-score   support

     family       0.96      1.00      0.98       102
    shelter       0.65      0.92      0.76        48
  volunteer       0.00      0.00      0.00         1
   donation       0.91      0.91      0.91        23
    service       0.00      0.00      0.00         1
      store       0.63      0.98      0.77        41
       food       0.00      0.00      0.00         3
     rescue       0.57      0.93      0.71        29
  community       1.00      0.12      0.22         8
      flood       0.00      0.00      0.00         1
       call       0.00      0.00      0.00         6
     school       0.50      1.00      0.67         5
      nurse       1.00      0.75      0.86         8
       home       0.74      1.00      0.85        31
     donate       0.00      0.00      0.00         1
       safe       0.00      0.00      0.00         1
     relief       0.75      0.92      0.83        13
       need       0.00      0.00      0.00   

In [677]:
# Get the model accuracy
accuracy_score(y_test, predictions)

0.8052287581699347